In [3]:
from PySide6.QtSql import QSqlDatabase
from mysql import connector

conn = connector.connect(    
    host='localhost',
    user='root',
    password='root')

cursor = conn.cursor()

# 检查并创建schema
cursor.execute("CREATE SCHEMA IF NOT EXISTS attendance_system")
cursor.execute("USE attendance_system")


In [5]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
for table in tables:
    print(table)
    cursor.execute(f"DESC {table[0]}")
    print(cursor.fetchall())
    print()
    

('course',)
[('course_name', 'varchar(100)', 'NO', 'PRI', None, ''), ('teacher_username', 'varchar(100)', 'NO', 'MUL', None, '')]

('course_student',)
[('student_id', 'int', 'YES', 'MUL', None, ''), ('course_name', 'varchar(100)', 'YES', 'MUL', None, '')]

('student',)
[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('gender', "enum('男','女')", 'YES', '', None, ''), ('age', 'char(3)', 'YES', '', None, ''), ('name', 'varchar(100)', 'YES', '', None, '')]

('teacher',)
[('name', 'varchar(100)', 'NO', '', None, ''), ('username', 'varchar(100)', 'NO', 'PRI', None, ''), ('password', 'varchar(100)', 'NO', '', None, '')]



In [4]:
# 开始事务
conn.start_transaction()

try:
    # Create the schema
    create_schema = "CREATE SCHEMA IF NOT EXISTS attendance_system"
    cursor.execute(create_schema)

    # Switch to the new schema
    cursor.execute("USE attendance_system")

    # Create the teacher table
    create_teacher_table = """
    CREATE TABLE IF NOT EXISTS teacher (
        name VARCHAR(100) NOT NULL,
        username VARCHAR(100) NOT NULL PRIMARY KEY,
        password VARCHAR(100) NOT NULL
    )
    """
    cursor.execute(create_teacher_table)

    # Create the student table
    create_student_table = """
    CREATE TABLE IF NOT EXISTS student (
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        gender ENUM('男', '女') NULL,
        age CHAR(3) NULL,
        name VARCHAR(100) NULL
    )
    """
    cursor.execute(create_student_table)

    # Create the course table
    create_course_table = """
    CREATE TABLE IF NOT EXISTS course (
        course_name VARCHAR(100) NOT NULL PRIMARY KEY,
        teacher_username VARCHAR(100) NOT NULL,
        FOREIGN KEY (teacher_username) REFERENCES teacher(username) On DELETE CASCADE
    )
    """
    cursor.execute(create_course_table)

    # Create the course_student table
    create_course_student_table = """
    CREATE TABLE IF NOT EXISTS course_student (
        course_name VARCHAR(100) NOT NULL,
        student_id INT NOT NULL,
        PRIMARY KEY (course_name, student_id),
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE,
        FOREIGN KEY (student_id) REFERENCES student(id) ON DELETE CASCADE
    )
    """
    cursor.execute(create_course_student_table)

    # Create the attendance_info table
    create_attendance_info_table = """
    CREATE TABLE IF NOT EXISTS attendance_info (
        date DATE NOT NULL,
        stu_id INT NOT NULL,
        course_name VARCHAR(100) NOT NULL,
        is_attendance BOOLEAN NOT NULL,
        PRIMARY KEY (date, stu_id, course_name),
        FOREIGN KEY (stu_id) REFERENCES student(id) ON DELETE CASCADE,
        FOREIGN KEY (course_name) REFERENCES course(course_name) ON DELETE CASCADE
    )
    """
    cursor.execute(create_attendance_info_table)

    # Commit the changes
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")

In [5]:
import random

# 随机生成老师和课程数据
teachers = []
courses = []

teacher_names = ["Alice", "Bob", "Charlie", "David", "Eva", "Frank", "Grace", "Hannah", "Ivy", "Jack", "Kathy", "Leo", "Mona", "Nina", "Oscar", "Paul", "Quincy", "Rachel", "Steve", "Tina", "Uma", "Victor", "Wendy", "Xander", "Yara", "Zane", "Aaron", "Bella", "Cindy", "Derek"]

for name in teacher_names:
    teachers.append((name, name, name))

for i in range(60):
    course_name = f"Course_{i+1}"
    teacher_username = random.choice(teacher_names)
    courses.append((course_name, teacher_username))

try:
    # 开始事务
    conn.start_transaction()

    # 插入老师数据
    cursor.executemany("INSERT INTO teacher (name, username, password) VALUES (%s, %s, %s)", teachers)

    # 插入课程数据
    cursor.executemany("INSERT INTO course (course_name, teacher_username) VALUES (%s, %s)", courses)

    # 提交事务
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")


In [6]:
import random

# 随机生成学生数据
students = []
genders = ['男', '女']
for i in range(1, 101):
    student = (i, random.choice(genders), str(random.randint(18, 25)), f'Student_{i}')
    students.append(student)

try:
    # 开始事务
    conn.start_transaction()

    # 插入学生数据
    cursor.executemany("INSERT INTO student (id, gender, age, name) VALUES (%s, %s, %s, %s)", students)

    # 随机选择30个学生填充course_student表
    for course in courses:
        selected_students = random.sample(students, 30)
        course_students = [(course[0], student[0]) for student in selected_students]
        cursor.executemany("INSERT INTO course_student (course_name, student_id) VALUES (%s, %s)", course_students)

    # 提交事务
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")

In [7]:
import datetime

# 定义日期范围
start_date = datetime.date(2025, 3, 26)
end_date = datetime.date(2025, 3, 30)
date_range = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1)]

attendance_data = []

try:
    # 开始事务
    conn.start_transaction()

    for date in date_range:
        # 每天随机选择20节课
        selected_courses = random.sample(courses, 20)
        for course in selected_courses:
            course_name = course[0]
            # 计算出勤率
            attendance_rate = random.uniform(0.8, 0.98)
            num_students = int(attendance_rate * len(selected_students))
            # 随机选择学生
            attending_students = random.sample(selected_students, num_students)
            for student in attending_students:
                attendance_data.append((date, student[0], course_name, True))
            # 剩下的学生标记为未出勤
            non_attending_students = [student for student in selected_students if student not in attending_students]
            for student in non_attending_students:
                attendance_data.append((date, student[0], course_name, False))

    # 插入出勤数据
    cursor.executemany("INSERT INTO attendance_info (date, stu_id, course_name, is_attendance) VALUES (%s, %s, %s, %s)", attendance_data)

    # 提交事务
    conn.commit()
except Exception as e:
    # 如果发生错误，回滚事务
    conn.rollback()
    print(f"An error occurred: {e}")

In [8]:
cursor.execute("""
SELECT total.date, total.count1, attend.count2, attend.count2 / total.count1
        FROM 
            (SELECT date, COUNT(*) AS count1
            FROM attendance_info
            WHERE course_name='course_12'
            GROUP BY date
            ) AS total
        JOIN
            (SELECT date, COUNT(*) AS count2
            FROM attendance_info
            WHERE course_name = 'course_12' AND is_attendance = TRUE
            GROUP BY date) as attend
        ON total.date = attend.date
        GROUP BY total.date;""")
print(cursor.fetchall())

[(datetime.date(2025, 3, 28), 30, 27, Decimal('0.9000')), (datetime.date(2025, 3, 30), 30, 28, Decimal('0.9333'))]
